<a href="https://colab.research.google.com/github/KiaroRB/data-analysis-projects/blob/main/bronze_data_ingestion_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
      .appName("bronze_ingestion")\
      .getOrCreate()